# COMS 4995_002 Deep Learning Assignment 2
Due on Monday, Oct 30, 11:59pm

This assignment can be done in groups of at most 3 students. Everyone must submit on Courseworks individually.

Write down the UNIs of your group (if applicable)

Member 1: Animesh Anant Sharma, aas2325

Member 2: Himanshu Aggarwal, ha2467

Member 3: Name, UNI

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import scipy.misc
import imageio
import glob
import sys
import tensorflow as tf
import os

import time, pickle


In [2]:
# Helper functions

def get_img_array(path):
    """
    Given path of image, returns it's numpy array
    """
    return imageio.imread(path)

def get_files(folder):
    """
    Given path to folder, returns list of files in it
    """
    filenames = [file for file in glob.glob(folder+'*/*')]
    filenames.sort()
    return filenames

def get_label(filepath, label2id):
    """
    Files are assumed to be labeled as: /path/to/file/999_frog.png
    Returns label for a filepath
    """
    tokens = filepath.split('/')
    label = tokens[-1].split('_')[1][:-4]
    if label in label2id:
        return label2id[label]
    else:
        sys.exit("Invalid label: " + label)

In [3]:
# Functions to load data

def get_labels(folder, label2id):
    """
    Returns vector of labels extracted from filenames of all files in folder
    :param folder: path to data folder
    :param label2id: mapping of text labels to numeric ids. (Eg: automobile -> 0)
    """
    files = get_files(folder)
    y = []
    for f in files:
        y.append(get_label(f,label2id))
    return np.array(y)

def one_hot(y, num_classes=10):
    """
    Converts each label index in y to vector with one_hot encoding
    """
    y_one_hot = np.zeros((y.shape[0], num_classes))
    a=range(0,y.shape[0])
    y_one_hot[a,y] = 1
    return y_one_hot

def get_label_mapping(label_file):
    """
    Returns mappings of label to index and index to label
    The input file has list of labels, each on a separate line.
    """
    with open(label_file, 'r') as f:
        id2label = f.readlines()
        id2label = [l.strip() for l in id2label]
    label2id = {}
    count = 0
    for label in id2label:
        label2id[label] = count
        count += 1
    return id2label, label2id

def get_images(folder):
    """
    returns numpy array of all samples in folder
    each column is a sample resized to 30x30 and flattened
    """
    files = get_files(folder)
    images = []
    count = 0
    
    for f in files:
        count += 1
        if count % 10000 == 0:
            print("Loaded {}/{}".format(count,len(files)))
        img_arr = get_img_array(f)
        img_arr = img_arr.flatten() / 255.0
        images.append(img_arr)
    X = np.column_stack(images)
    return X

def get_train_data(data_root_path):
    """
    Return X and y
    """
    train_data_path = data_root_path + 'train'
    id2label, label2id = get_label_mapping(data_root_path+'labels.txt')
    print(label2id)
    X = get_images(train_data_path)
    y = get_labels(train_data_path, label2id)
    return X, y

def save_predictions(filename, y):
    """
    Dumps y into .npy file
    """
    np.save(filename, y)

In [4]:
# Load the data
data_root_path = '/home/animesh/deeplearning/cifar10-hw1/'
X1, y = get_train_data(data_root_path) # this may take a few minutes
X_test1 = get_images(data_root_path + 'test')
print('Data loading done')

{'frog': 6, 'dog': 5, 'cat': 3, 'horse': 7, 'truck': 9, 'ship': 8, 'airplane': 0, 'deer': 4, 'bird': 2, 'automobile': 1}
Loaded 10000/50000
Loaded 20000/50000
Loaded 30000/50000
Loaded 40000/50000
Loaded 50000/50000
Loaded 10000/10000
Data loading done


In [5]:
X = X1.astype('float32')
X_test = X_test1.astype('float32')
X = X.T


#training set
X_train = X[:45000]
#mean of training dataset
me = np.mean(X_train, axis = 0)
#std of training dataset
st = np.std(X_train, axis = 0)
#training dataset normalization
X_train -= me
X_train /= st
y_train = y[:45000]
Y_train = one_hot(y_train, 10)


#validation set
X_val = X[45000:]
#validation set normalization
X_val -= me
X_val /= st
y_val = y[45000:]
Y_val = one_hot(y_val, 10)


#test dataset
X_test = X_test.T
#test set normalization
X_test -= me
X_test /= st


#regularizer definition
regularizer = tf.contrib.layers.l2_regularizer(scale=0.01)

##  Simple convolutional neural network


In [6]:
def conv_layer1(input, size_in, size_out, name="conv1"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=5e-2), name="W")
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        b = tf.Variable(tf.constant(0.0, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    
def conv_layer2(input, size_in, size_out, name="conv2"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([5, 5, size_in, size_out], stddev=5e-2), name="W")
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        b = tf.Variable(tf.constant(0.0, shape=[size_out]), name="B")
        conv = tf.nn.conv2d(input, w, strides=[1, 1, 1, 1], padding="SAME")
        act = tf.nn.relu(conv + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act

def fc_layer1(input, size_in, size_out, name="fc1"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.nn.relu(tf.matmul(input, w) + b)
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    
def fc_layer2(input, size_in, size_out, name="fc2"):
    with tf.name_scope(name):
        w = tf.Variable(tf.truncated_normal([size_in, size_out], stddev=0.1), name="W")
        tf.add_to_collection(tf.GraphKeys.REGULARIZATION_LOSSES, w)
        b = tf.Variable(tf.constant(0.1, shape=[size_out]), name="B")
        act = tf.matmul(input, w) + b
        tf.summary.histogram("weights", w)
        tf.summary.histogram("biases", b)
        tf.summary.histogram("activations", act)
        return act
    
def get_batch(X, y, batch_size):
        """
        Return minibatch of samples and labels
        
        :param X, y: samples and corresponding labels
        :parma batch_size: minibatch size
        :returns: (tuple) X_batch, y_batch
        """
        temp = np.random.choice(range(45000), batch_size, replace=False)
        X_batch = X[temp]
        y_batch = y[temp]

        return X_batch, y_batch 

In [7]:
def cifar_model(learning_rate, b_s,epochs):
    tf.reset_default_graph()
    sess = tf.Session()

    # Setup placeholders
    x = tf.placeholder(tf.float32, shape=[None,3072], name="x")
    x_image = tf.reshape(x, [-1, 32, 32, 3])
    y = tf.placeholder(tf.float32, shape=[None,10], name="labels")

    #conv1
    conv1 = conv_layer1(x_image, 3, 16, "conv1")
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                            padding='SAME', name='pool1')
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                        name='norm1')
    
    #conv2
    conv2 = conv_layer2(norm1, 16, 32, "conv2")
    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                        name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 2, 2, 1],
                             strides=[1, 2, 2, 1], padding='SAME', name='pool2')

    flattened = tf.contrib.layers.flatten(pool2)
    dim = flattened.get_shape()[1].value

    fc1 = fc_layer1(flattened, dim, 256, "fc1")
    logits = fc_layer2(fc1, 256, 10, "fc2")

    with tf.name_scope("xent"):
        reg_variables = tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)
        reg_term = tf.contrib.layers.apply_regularization(regularizer, reg_variables)
        xent = tf.reduce_mean(
            tf.nn.softmax_cross_entropy_with_logits(
                logits=logits, labels=y), name="xent")
        xent += reg_term
        tf.summary.scalar("xent", xent)
        
    #with tf.name_scope("train"):
    #    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(xent)
        
    with tf.name_scope('train'):
        optimizer = tf.train.GradientDescentOptimizer(learning_rate)
        # Get the gradient pairs (Tensor, Variable)
        grads = optimizer.compute_gradients(xent)
        # Update the weights wrt to the gradient
        train_step = optimizer.apply_gradients(grads)
        # Save the grads with tf.summary.histogram
        for index, grad in enumerate(grads):
            tf.summary.histogram("{}-grad".format(grads[index][1].name.replace(':','_')), grads[index])

    with tf.name_scope("accuracy"):
        correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(y, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
        tf.summary.scalar("accuracy", accuracy)

    summ = tf.summary.merge_all()
    
    saver = tf.train.Saver()
    sess.run(tf.global_variables_initializer())
    writer_train = tf.summary.FileWriter('/tmp/cifar10_train1')
    writer_train.add_graph(sess.graph)
    writer_val = tf.summary.FileWriter('/tmp/cifar10_val1')
    writer_val.add_graph(sess.graph)
    
    for i in range(epochs+1):
        example_batch, label_batch = get_batch(X_train, Y_train, b_s)
        if i % 500 == 0:
            [train_accuracy, s, train_loss] = sess.run([accuracy, summ, xent], feed_dict={x: example_batch, y: label_batch})
            writer_train.add_summary(s, i)
            print ("epoch:%i, train loss:%.2f, train accuracy:%.2f" % (i,train_loss,train_accuracy))
        if i % 500 == 0:
            [val_accuracy, s, val_loss] = sess.run([accuracy, summ, xent], feed_dict={x: X_val, y: Y_val})
            print ("epoch:%i, val loss:%.2f, val accuracy:%.2f" % (i,val_loss,val_accuracy))
            writer_val.add_summary(s, i)
            #saver.save(sess, os.path.join('/tmp/cifar10_1', "model.ckpt"), i)
        sess.run(train_step, feed_dict={x: example_batch, y: label_batch})
        
    y_predicted = sess.run(logits, feed_dict={x: X_test})
    save_predictions('ans1-aas2325', y_predicted.T)

In [8]:
def main():
    learning_rate = 0.05
    batch_size = 256
    epochs = 5000
    cifar_model(learning_rate, batch_size, epochs)


if __name__ == '__main__':
    main()

epoch:0, train loss:22.94, train accuracy:0.10
epoch:0, val loss:22.91, val accuracy:0.11
epoch:500, train loss:13.81, train accuracy:0.54
epoch:500, val loss:13.76, val accuracy:0.53
epoch:1000, train loss:8.66, train accuracy:0.65
epoch:1000, val loss:8.77, val accuracy:0.59
epoch:1500, train loss:5.64, train accuracy:0.69
epoch:1500, val loss:5.80, val accuracy:0.61
epoch:2000, train loss:3.87, train accuracy:0.70
epoch:2000, val loss:3.94, val accuracy:0.66
epoch:2500, train loss:2.88, train accuracy:0.64
epoch:2500, val loss:2.94, val accuracy:0.64
epoch:3000, train loss:2.06, train accuracy:0.74
epoch:3000, val loss:2.19, val accuracy:0.69
epoch:3500, train loss:1.50, train accuracy:0.83
epoch:3500, val loss:1.79, val accuracy:0.70
epoch:4000, train loss:1.22, train accuracy:0.84
epoch:4000, val loss:1.52, val accuracy:0.72
epoch:4500, train loss:1.20, train accuracy:0.81
epoch:4500, val loss:1.41, val accuracy:0.71
epoch:5000, train loss:1.20, train accuracy:0.78
epoch:5000, val

In [9]:
## test if your numpy file has been saved correctly
loaded_y = np.load('ans1-aas2325.npy')
print(loaded_y.shape)
loaded_y[:,:10]

(10, 10000)


array([[-0.7495687 ,  4.82834864,  4.1027751 , -0.94138175, -1.72743773,
         4.47856808,  1.01133513,  0.67918223,  4.71195889, -2.05389953],
       [-2.24105525,  7.765769  , -4.03055573, -5.51350307, -3.7088244 ,
         1.84614789,  1.14905512, -4.94488287, -0.4304924 ,  9.82844925],
       [-1.07777417, -3.36657786,  1.72933006,  0.71412218,  0.13018219,
         0.43590462, -2.13995934,  2.11371636,  1.29010057, -2.59998465],
       [ 5.18138695, -1.81094372,  1.03561103,  1.13204324,  3.85618687,
        -1.88940275,  1.15162277,  0.57707816, -0.50278556, -0.89251268],
       [-1.3155899 , -3.33123946,  4.15081453,  3.91557717,  0.19859576,
        -0.01313931,  1.18661761,  5.11981392, -1.47888017, -2.14182758],
       [ 3.40518117, -5.85264349,  0.57073253,  2.73257089,  5.67234802,
        -3.98332191, -1.04266489,  1.13203692, -3.17853761, -2.32168841],
       [-0.3470999 , -3.89025617, -3.26397824, -1.4531883 , -1.62593174,
        -1.97059643, -2.78255582, -1.47051108